Generate a random float.

In [6]:
import random
import struct
import numpy as np
import torch
import qtorch
from qtorch.quant import posit_quantize
from qtorch.quant import convert_to_posit

testfloat=random.random()*1e3
print(testfloat)

896.4925589286798


Extract sign, exponent and fraction

In [7]:
def float_to_bits(f):
    # Pack float into 8 bytes, then unpack as 64-bit unsigned int
    packed = struct.pack('>d', f)  # Big endian double
    integer_representation = int.from_bytes(packed, byteorder='big')
    return integer_representation

def extract_components(f):
    bits = float_to_bits(f)

    # Extract sign (1 bit), exponent (11 bits), and fraction (52 bits)
    sign = (bits >> 63) & 0x1
    exponent = (bits >> 52) & 0x7FF
    fraction = bits & ((1 << 52) - 1)

    # Compute values
    sign_val = (-1) ** sign
    exponent_val = exponent - 1023  # Bias for double-precision is 1023
    print(type(1 * (2 ** -(1 + 1))))
    fraction_val = 1 + sum(
        [((fraction & (1 << (52 - i))) >> (51 - i)) * (1 * (2 ** -(i + 1))) for i in range(52)]
    ) if exponent != 0 else sum(
        [((fraction & (1 << (52 - i))) >> (51 - i)) * 1 * (2 ** -(i + 1)) for i in range(52)]
    )

    # Format binary strings
    sign_bits = f"{sign:b}"
    exponent_bits = f"{exponent:011b}"
    fraction_bits = f"{fraction:052b}"

    print(f"Input float: {f}")
    print(f"Sign bit     : {sign_bits} (value: {sign_val})")
    print(f"Exponent bits: {exponent_bits} (value: {exponent} => unbiased: {exponent_val})")
    print(f"Fraction bits: {fraction_bits}")
    print(f"Fraction value: {fraction_val}")
    print(f"Reconstructed value (approx): {sign_val * (2 ** exponent_val) * fraction_val}")

extract_components(testfloat)


<class 'float'>
Input float: 896.4925589286798
Sign bit     : 0 (value: 1)
Exponent bits: 10000001000 (value: 1032 => unbiased: 9)
Fraction bits: 1100000000111111000011000010101111000101000001000011
Fraction value: 1.7509620291575776
Reconstructed value (approx): 896.4925589286797


Convert float value to posit.

In [8]:
n = 16
e = 2
float_tensor = torch.tensor(testfloat, dtype=torch.float)
b = (convert_to_posit(float_tensor, nsize=n, es=e)).item()
posit_bit_string = bin(b)
print(b)
print(posit_bit_string)

29568
0b111001110000000


Extract sign, regime, exponent and fraction from posit.

In [9]:
#Function to count leading ones
def count_leading_ones(n):
    binary_str = bin(n)[2:]
    count = 0
    for digit in binary_str:
        if digit == '1':
            count += 1
        else:
            break
    return count

In [10]:
sign = b & (1 << n)
posit_without_sign = np.uint16(b << 1)
regime_bits = count_leading_ones(posit_without_sign)
posit_without_regime = np.uint16(posit_without_sign << (regime_bits + 1))
exponent_bits = posit_without_regime >> (n - e)
fraction_size = n - regime_bits - 1 - e
fraction_bits = (posit_without_regime << 2) >> (n - fraction_size)
fraction = 1 + (fraction_bits / 2**fraction_size)
useed = 2**(2**e)
posit_value = useed**(regime_bits - 1) * 2**exponent_bits
posit_value = posit_value * fraction
total_exponent = (2**e) * (regime_bits - 1) + exponent_bits

print(f"sign: {sign}")
print(f"regime: {regime_bits - 1}")
print(f"exponent: {exponent_bits}")
print(f"total exponent: {total_exponent}")
print(f"fraction: {fraction}")
print(f"posit value: {posit_value:.7f}")

sign: 0
regime: 2
exponent: 1
total exponent: 9
fraction: 1.75
posit value: 896.0000000
